# Analyzing the influence of the "FUN bug" on the CLM5-PPE
- Updated Sept 7, 2023
- Daniel Kennedy, djk2120@ucar.edu

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import glob
import matplotlib.pyplot as plt
import seaborn as sns

## spin up some extra cores on CASPER

In [2]:
# Setup your PBSCluster
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client
ncores=1
nmem='10GB'
cluster = PBSCluster(
    cores=ncores, # The number of cores you want
    memory=nmem, # Amount of memory
    processes=1, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='$TMPDIR', # Use your local directory
    resource_spec='select=1:ncpus='+str(ncores)+':mem='+nmem, # Specify resources
    account='P93300641', # Input your project ID here
    walltime='00:25:00', # Amount of wall time
    interface='ib0', # Interface to use
)

# Scale up
cluster.scale(30)

# Setup your client
client = Client(cluster)

In [2]:
from utils import *

### FUNbug in the OAAT

In [3]:
la=xr.open_dataset('sparsegrid_landarea.nc').landarea

In [50]:
files=['/glade/scratch/djk2120/PPEn11_CTL2010_FUNbug0001/run/PPEn11_CTL2010_FUNbug0001.clm2.h0.2005-02-01-00000.nc',
       '/glade/campaign/asp/djk2120/PPEn11/CTL2010/n08/PPEn08_CTL2010_OAAT0000.clm2.h0.2005-02-01-00000.nc']

In [51]:
dvs=['GPP','AR','HR','TLAI','TOTVEGC','TOTSOMC','EFLX_LH_TOT','SNOWDP','SOILWATER_10CM']
ds=get_ds(files,['ens'],dvs=dvs)

In [52]:
for v in dvs:
    x=gmean(amean(ds[v]),la).mean(dim='year')
    dx=x[0]/x[1]
    p=''
    if dx>1:p='+'
    print(v,p+str(np.round(100*dx.values-100,1))+'%')

GPP -3.6%
AR -0.1%
HR -8.2%
TLAI -5.8%
TOTVEGC -1.8%
TOTSOMC -4.6%
EFLX_LH_TOT -1.1%
SNOWDP +0.0%
SOILWATER_10CM +0.6%


### FUNbug in the LHC

In [39]:
files=['/glade/scratch/linnia/archive/PPEn11_transient_FUNbug0001/lnd/hist/PPEn11_transient_FUNbug0001.clm2.h0.2010-02-01-00000.nc',
       '/glade/scratch/djk2120/PPEn11_transient_LHC0000/run/PPEn11_transient_LHC0000.clm2.h0.2010-02-01-00000.nc']

In [40]:
dvs=['GPP','AR','HR','TLAI','TOTVEGC','TOTSOMC','EFLX_LH_TOT','SNOWDP','SOILWATER_10CM']
ds=get_ds(files,['ens'],dvs=dvs)

In [48]:
for v in dvs:
    x=gmean(amean(ds[v]),la).mean(dim='year')
    dx=x[0]/x[1]
    p=''
    if dx>1:p='+'
    print(v,p+str(np.round(100*dx.values-100,1))+'%')

GPP -5.0%
AR -2.3%
HR -9.7%
TLAI -6.5%
TOTVEGC -2.4%
TOTSOMC -5.1%
EFLX_LH_TOT -1.5%
SNOWDP +0.0%
SOILWATER_10CM +0.5%
